Comando para rodar o GraphHopper no terminal - atenção para o PBF a ser carregado:

**PBF da Rede 2019; custom model ajustado (LTS)**

clear && cd /home/livre/Desktop/Base_GtsRegionais/GitLab/yellow_src/graphhopper/ && rm -rf graph-cache/ && java -Ddw.graphhopper.datareader.file=/home/livre/Desktop/Base_GtsRegionais/GitLab/yellow_dados/07_graphhopper/03_PBFs_SP_rede_2019/20220216_sao_paulo_edited_20230521_A_infraciclo_atual.osm.pbf -jar graphhopper/web/target/graphhopper-web-*.jar server graphhopper/config-example_LTS.yml

In [1]:
# carregar bibliotecas
suppressPackageStartupMessages(library('tidyverse'))
suppressPackageStartupMessages(library('tidylog'))
suppressPackageStartupMessages(library('httr'))
suppressPackageStartupMessages(library('jsonlite'))

In [2]:
# Aplicar funcoes em paralelo
library('future.apply')
# Checando: Jupyter suporta multicore?
future::supportsMulticore()

Loading required package: future



[1] TRUE

In [3]:
# https://stackoverflow.com/questions/40536067/how-to-adjust-future-global-maxsize
# For 850MB: 850*1024^2 = 891289600
# For +1.3GB: 1500*1024^2 = 1572864000
# options(future.globals.maxSize = 891289600)
options(future.globals.maxSize = 5e6) # 5 MB
options(future.globals.maxSize = 15*1024^2) # 15 MB
parallelly::availableCores()

system 
     8

In [4]:
options(parallelly.availableCores.methods = "mc.cores")
options(mc.cores = 6)
parallelly::availableCores()

mc.cores 
       6

In [5]:
# Estrutura de pastas
pasta_ssd         <- "/media/livre/SSD120GB/yellow"
pasta_dados       <- "../../yellow_dados"
# dados_originais   <- sprintf("%s/00_dados_originais/IPEA", pasta_dados)
pasta_aop_rev     <- sprintf("%s/12_aop_revisitado", pasta_dados)
# pasta_aoprev_hex  <- sprintf("%s/01_hexagonos_26_vizinhos", pasta_aop_rev)
pasta_aoprv_alter <- sprintf("%s/03_alternatives_2019_2028", pasta_aop_rev)
pasta_ids_aopt_19 <- sprintf("%s/A_2019_osm_way_ids_aop", pasta_ssd)
pasta_rts_aopt_19 <- sprintf("%s/B_2019_rotas_modeladas_alternatives", pasta_ssd)
dir.create(pasta_ids_aopt_19, recursive = TRUE, showWarnings = FALSE)
dir.create(pasta_rts_aopt_19, recursive = TRUE, showWarnings = FALSE)

# Método 5

In [6]:
# ------------------------------------------------------------------------------
# Routing a partir de dois pontos com rotas alternativas (até 3 por par OD)
# ------------------------------------------------------------------------------

# Faz query de routing no GraphHopper e retorna resultados principais em dataframe,
# com rotas até 3 alternativas por par OD - aceita um dataframe de uma linha como
# entrada
gh_route_alt <- function(hex_id, route_options) {
  # url <- 'http://localhost:8989/route/?point=-23.5314933121698%2C-46.634354542765&point=-23.5390199310058%2C-46.6376369484305&profile=bike&instructions=false&calc_points=true&details=average_speed'
  # url <- ods_vgs %>% slice(1) %>% select(url) %>% pull()
  # Estação Vila Madalena: -23.546258,-46.690898
  # IME: -23.559007,-46.73208
  # CCSP: -23.571498,-46.639806
  
  # df_line$url <- paste('http://localhost:8989/route/?point=', 
  # '-23.541504', '%2C', '-46.685779', '&point=', 
  # '-23.571498', '%2C', '-46.639806', route_options,
  # sep = '')
  
  # hex_id <- '89a81046b2fffff-89a81044d8fffff'
  # hex_id <- hex_com_vizinhos %>% head(1) %>% select(id) %>% pull()
  df_line <- hex_com_vizinhos %>% filter(id == hex_id)
  # Encurtar hex_id - todos aqui são '89a81' + 6 caracteres de dígito ou letra = 'ffff'
  hex_id_short <- str_replace(hex_id, '^89a81([a-z0-9]{6})ffff-89a81([a-z0-9]{6})ffff', '\\1-\\2')
  hex_id_base  <- str_sub(hex_id_short, 1, 6)
  
  # Fazer a GET de roteamento no Grahphopper
  # print(df_line$url)
  gh_response <- GET(df_line$url)
  
  # Mensagem tem que ser "Success: (200) OK"
  if (http_status(gh_response)$message == 'Success: (200) OK') {
    
    # Resposta da query, já colapsada e transformada em dataframe
    # Remover aviso de 'No encoding supplied: defaulting to UTF-8' na linha fromJSON()
    suppressMessages(
      response_text <- 
        # Ignorar aviso 'argument is not an atomic vector; coercing'
        suppressWarnings(str_c(content(gh_response, 'text'), collapse = ", ")) %>% 
        # Concatenar toda a string de resultados
        str_c("[", ., "]") %>% 
        # Transformar em dataframe
        fromJSON() %>% 
        as.data.frame()
    )
    
    # Nos interessa a coluna de 'paths', como um novo dataframe
    paths <- response_text$paths %>% as.data.frame()
    
    # Puxar osm_way_ids dos resultados de cada alternativa e gravar em pasta separada
    for (i in seq(1, length(paths$details$osm_way_id))) {
      # i <- 1
      osm_ways <- paths$details$osm_way_id[i] %>% as.data.frame()
      
      if (nrow(osm_ways) > 0) {
        # Manter somente osm_ids e inserir número da rota alternativa
        osm_ways <- osm_ways %>% select(osm_way_id = X3) %>% mutate(hex_id = hex_id_short,
                                                                    alt = i,
                                                                    index = row_number(),
                                                                    .before = 'osm_way_id')
        # Gravar resultados agrupados por hex_id_short de origem
        osm_way_out <- sprintf('%s/%s.csv', pasta_ids_aopt_19, hex_id_base)
        if (file.exists(osm_way_out)) {
          write_delim(osm_ways, osm_way_out, delim = ';', append = TRUE)
        } else {
          write_delim(osm_ways, osm_way_out, delim = ';', append = FALSE)
        }
        
      } else {
        # Se não há osm_way_ids, é porque os pontos estão muito próximos uns dos
        # outros, mesmo que seja um osm_way_id diferente entre a origem e o 
        # destino. A distância e a velocidade calculadas vão ser zero também - 
        # pular este registro, que vai ser vazio
        return(sprintf('Pulando: %s não tem osm_way_ids (provavelmente tem distância = 0)', hex_id))
      }
      
    }
    
    
    # Isolar colunas de interesse
    paths <- 
      paths %>% 
      # Calcular tempo em segundos e velocidade média
      mutate(time = time / 1000,
             speed = distance / time * 3.6) %>% 
      # Descartar colunas extras - a coluna poly é o shape da rota traçada
      select(distance, weight, time, speed, poly = points)
    
    # Testar polyline:
    # https://valhalla.github.io/demos/polyline/?unescape=true&polyline6=false#%0A
    
    # Adicionar colunas de informação vindas do dataframe original
    paths <- 
      paths %>% 
      mutate(hex_id    = hex_id_short,
             alt       = row_number(),
             # id_hex.x  = df_line$id_hex_x,
             # id_hex.y  = df_line$id_hex_y,
             .before = 'distance') #%>% 
    # mutate(lon.x     = df_line$lon.x,
    #        lat.x     = df_line$lat.x,
    #        lon.y     = df_line$lon.y,
    #        lat.y     = df_line$lat.y,
    #        .after = 'poly')
    
  } else {
    
    # Se a query no GraphHopper não deu resultados, guardar como dataframe vazio
    paths <- data.frame(hex_id    = hex_id_short,
                        alt       = NA,
                        # id_hex.x  = df_line$id_hex_x,
                        # id_hex.y  = df_line$id_hex_y,
                        distance  = NA,
                        weight    = NA,
                        time      = NA,
                        speed     = NA,
                        poly      = NA
                        # lon.x     = df_line$lon.x,
                        # lat.x     = df_line$lat.x,
                        # lon.y     = df_line$lon.y,
                        # lat.y     = df_line$lat.y
    )
    
  }
  
  # Guardar resultados temporários agrupados por hex_id_short de origem
  tmp_file <- sprintf('%s/%s_modalt.csv', pasta_rts_aopt_19, hex_id_base)
  if (file.exists(tmp_file)) {
    write_delim(paths, tmp_file, delim = ';', append = TRUE)
  } else {
    write_delim(paths, tmp_file, delim = ';', append = FALSE)
  }
  
  # Guardar ids já processados em arquivo próprio
  df_line <- df_line %>% select(id)
  ids_processados <- sprintf('%s/tmp_00_ids_processados_2019.csv', pasta_ssd)

  if (file.exists(ids_processados)) {
    write_delim(df_line, ids_processados, delim = ';', append = TRUE)
  } else {
    write_delim(df_line, ids_processados, delim = ';', append = FALSE)
  }
  
}

In [7]:
detach("package:tidylog")

In [8]:
# Arquivos a processar
arqs <- list.files(pasta_ssd, pattern = '^tmp_base_para_routing_res09_26vizinhos_[0-9]{3}.csv', full.names = TRUE)
# Arquivos já processados
ids_processados <- sprintf('%s/tmp_00_ids_processados_2019.csv', pasta_ssd)

for (arq in arqs) {
  # arq <- arqs[1]
  print(arq)
  
  # Abrir base para routing com 26 vizinhos, manter só id e url  
  hex_com_vizinhos <- read_delim(arq, delim = ';', col_types = cols(.default = "c"))
  
  # Checar quais resultados já foram rodados - abrir lista, puxar ids e remover
  # do dataframe hex_com_vizinhos se houver
  # library('tidylog')
  if (file.exists(ids_processados)) {
    arqs_resultados <- read_delim(ids_processados, delim = ';', col_types = "c")
    hex_com_vizinhos <- hex_com_vizinhos %>% filter(!id %in% arqs_resultados$hex_id)
    rm(arqs_resultados)
    print(nrow(hex_com_vizinhos))
  }
  
  # Se arquivo já foi completamente processado, removê-lo e passar para o seguinte
  if (nrow(hex_com_vizinhos) == 0) { 
    file.remove(arq) 
    next
  }
  
  # Limpar memória
  gc(T)
  
  # Criar ttmatrix a partir do GrahHopper - melhor rodar no Jupyter se for AOP;
  # se forem só as rotas originais, é ok rodar no RStudio
  # for (id in hex_com_vizinhos$id) { gh_route_alt(id, route_options = route_options) }
  # Rodar função para todos os arquivos- multi thread (Jupyter)
  (start = Sys.time())
  future::plan(future::multicore)
  invisible(future.apply::future_lapply(X   = hex_com_vizinhos$id,
                                        FUN = gh_route_alt,
                                        future.seed = TRUE))
  Sys.time()
  Sys.time() - start
  
  # Remover arquivo inteiramente processado
  file.remove(arq) 
  
  # Limpar memória
  rm(hex_com_vizinhos)  
  gc(T)
  
}

[1] "/media/livre/SSD120GB/yellow/tmp_base_para_routing_res09_26vizinhos_114.csv"
[1] 12270
[1] "/media/livre/SSD120GB/yellow/tmp_base_para_routing_res09_26vizinhos_115.csv"
[1] 18033
[1] "/media/livre/SSD120GB/yellow/tmp_base_para_routing_res09_26vizinhos_116.csv"
[1] 21281
[1] "/media/livre/SSD120GB/yellow/tmp_base_para_routing_res09_26vizinhos_117.csv"
[1] 16758
[1] "/media/livre/SSD120GB/yellow/tmp_base_para_routing_res09_26vizinhos_118.csv"
[1] 22856
[1] "/media/livre/SSD120GB/yellow/tmp_base_para_routing_res09_26vizinhos_119.csv"
[1] 19895
[1] "/media/livre/SSD120GB/yellow/tmp_base_para_routing_res09_26vizinhos_120.csv"
[1] 22144
[1] "/media/livre/SSD120GB/yellow/tmp_base_para_routing_res09_26vizinhos_121.csv"
[1] 18935
[1] "/media/livre/SSD120GB/yellow/tmp_base_para_routing_res09_26vizinhos_122.csv"
[1] 18969
[1] "/media/livre/SSD120GB/yellow/tmp_base_para_routing_res09_26vizinhos_123.csv"
[1] 23492
[1] "/media/livre/SSD120GB/yellow/tmp_base_para_routing_res09_26vizinhos_124.csv

# Método 4

In [9]:
# # ------------------------------------------------------------------------------
# # Routing a partir de dois pontos com rotas alternativas (até 3 por par OD)
# # ------------------------------------------------------------------------------

# # Faz query de routing no GraphHopper e retorna resultados principais em dataframe,
# # com rotas até 3 alternativas por par OD - aceita um dataframe de uma linha como
# # entrada
# gh_route_alt <- function(hex_id, route_options) {
#   # url <- 'http://localhost:8989/route/?point=-23.5314933121698%2C-46.634354542765&point=-23.5390199310058%2C-46.6376369484305&profile=bike&instructions=false&calc_points=true&details=average_speed'
#   # url <- ods_vgs %>% slice(1) %>% select(url) %>% pull()
#   # Estação Vila Madalena: -23.546258,-46.690898
#   # IME: -23.559007,-46.73208
#   # CCSP: -23.571498,-46.639806
  
#   # df_line$url <- paste('http://localhost:8989/route/?point=', 
#   # '-23.541504', '%2C', '-46.685779', '&point=', 
#   # '-23.571498', '%2C', '-46.639806', route_options,
#   # sep = '')
  
#   # hex_id <- '89a81046b2fffff-89a81044d8fffff'
#   # hex_id <- hex_com_vizinhos %>% head(1) %>% select(id) %>% pull()
#   df_line <- hex_com_vizinhos %>% filter(id == hex_id)
  
#   # Fazer a GET de roteamento no Grahphopper
#   # print(df_line$url)
#   gh_response <- GET(df_line$url)
  
#   # Mensagem tem que ser "Success: (200) OK"
#   if (http_status(gh_response)$message == 'Success: (200) OK') {
    
#     # Resposta da query, já colapsada e transformada em dataframe
#     # Remover aviso de 'No encoding supplied: defaulting to UTF-8' na linha fromJSON()
#     suppressMessages(
#       response_text <- 
#         # Ignorar aviso 'argument is not an atomic vector; coercing'
#         suppressWarnings(str_c(content(gh_response, 'text'), collapse = ", ")) %>% 
#         # Concatenar toda a string de resultados
#         str_c("[", ., "]") %>% 
#         # Transformar em dataframe
#         fromJSON() %>% 
#         as.data.frame()
#     )
    
#     # Nos interessa a coluna de 'paths', como um novo dataframe
#     paths <- response_text$paths %>% as.data.frame()
    
#     # Puxar osm_way_ids dos resultados de cada alternativa e gravar em pasta separada
#     for (i in seq(1, length(paths$details$osm_way_id))) {
#       # i <- 1
#       osm_ways <- paths$details$osm_way_id[i] %>% as.data.frame()
      
#       if (nrow(osm_ways) > 0) {
#         # Manter somente osm_ids e inserir número da rota alternativa
#         osm_ways <- osm_ways %>% select(osm_way_id = X3) %>% mutate(alt = i,
#                                                                     index = row_number(),
#                                                                     .before = 'osm_way_id')
#         # Gravar resultados
#         osm_way_out <- sprintf('%s/%s.csv', pasta_ids_aopt_19, df_line$id)
#         if (file.exists(osm_way_out)) {
#           write_delim(osm_ways, osm_way_out, delim = ';', append = TRUE)
#         } else {
#           write_delim(osm_ways, osm_way_out, delim = ';', append = FALSE)
#         }
        
#       } else {
#         # Se não há osm_way_ids, é porque os pontos estão muito próximos uns dos
#         # outros, mesmo que seja um osm_way_id diferente entre a origem e o 
#         # destino. A distância e a velocidade calculadas vão ser zero também - 
#         # pular este registro, que vai ser vazio
#         return(sprintf('Pulando: %s não tem osm_way_ids (provavelmente tem distância = 0)', hex_id))
#       }
      
#     }
    
    
#     # Isolar colunas de interesse
#     paths <- 
#       paths %>% 
#       # Calcular tempo em segundos e velocidade média
#       mutate(time = time / 1000,
#              speed = distance / time * 3.6) %>% 
#       # Descartar colunas extras - a coluna poly é o shape da rota traçada
#       select(distance, weight, time, speed, poly = points)
    
#     # Testar polyline:
#     # https://valhalla.github.io/demos/polyline/?unescape=true&polyline6=false#%0A
    
#     # Adicionar colunas de informação vindas do dataframe original
#     paths <- 
#       paths %>% 
#       mutate(hex_id    = df_line$id,
#              alt       = row_number(),
#              # id_hex.x  = df_line$id_hex_x,
#              # id_hex.y  = df_line$id_hex_y,
#              .before = 'distance') #%>% 
#     # mutate(lon.x     = df_line$lon.x,
#     #        lat.x     = df_line$lat.x,
#     #        lon.y     = df_line$lon.y,
#     #        lat.y     = df_line$lat.y,
#     #        .after = 'poly')
    
#   } else {
    
#     # Se a query no GraphHopper não deu resultados, guardar como dataframe vazio
#     paths <- data.frame(hex_id    = df_line$id,
#                         alt       = NA,
#                         # id_hex.x  = df_line$id_hex_x,
#                         # id_hex.y  = df_line$id_hex_y,
#                         distance  = NA,
#                         weight    = NA,
#                         time      = NA,
#                         speed     = NA,
#                         poly      = NA
#                         # lon.x     = df_line$lon.x,
#                         # lat.x     = df_line$lat.x,
#                         # lon.y     = df_line$lon.y,
#                         # lat.y     = df_line$lat.y
#     )
    
#   }
  
#   # Guardar resultados temporários
#   tmp_file <- sprintf('%s/%s_modalt.csv', pasta_rts_aopt_19, df_line$id)
#   write_delim(paths, tmp_file, delim = ';')
  
#   # Guardar ids já processados em arquivo próprio
#   df_line <- df_line %>% select(id)
#   ids_processados <- sprintf('%s/tmp_00_ids_processados.csv', pasta_aoprv_alter)

#   if (file.exists(ids_processados)) {
#     write_delim(df_line, ids_processados, delim = ';', append = TRUE)
#   } else {
#     write_delim(df_line, ids_processados, delim = ';', append = FALSE)
#   }
  
# }

In [10]:
# detach("package:tidylog")

In [11]:
# # Arquivos a processar
# arqs <- list.files(pasta_aoprv_alter, pattern = '^tmp_base_para_routing_res09_26vizinhos_[0-9]{3}.csv', full.names = TRUE)
# # Arquivos já processados
# ids_processados <- sprintf('%s/tmp_00_ids_processados.csv', pasta_aoprv_alter)

# for (arq in arqs) {
#   # arq <- arqs[1]
#   print(arq)
  
#   # Abrir base para routing com 26 vizinhos, manter só id e url  
#   hex_com_vizinhos <- read_delim(arq, delim = ';', col_types = cols(.default = "c"))
  
#   # Checar quais resultados já foram rodados - abrir lista, puxar ids e remover
#   # do dataframe hex_com_vizinhos se houver
#   # library('tidylog')
#   if (file.exists(ids_processados)) {
#     arqs_resultados <- read_delim(ids_processados, delim = ';', col_types = "c")
#     hex_com_vizinhos <- hex_com_vizinhos %>% filter(!id %in% arqs_resultados$id)
#     rm(arqs_resultados)
#     nrow(hex_com_vizinhos)
#   }
  
#   # Se arquivo já foi completamente processado, removê-lo e passar para o seguinte
#   if (nrow(hex_com_vizinhos) == 0) { 
#     file.remove(arq) 
#     next
#   }
  
#   # Limpar memória
#   gc(T)
  
#   # Criar ttmatrix a partir do GrahHopper - melhor rodar no Jupyter se for AOP;
#   # se forem só as rotas originais, é ok rodar no RStudio
#   # for (id in hex_com_vizinhos$id) { gh_route_alt(id, route_options = route_options) }
#   # Rodar função para todos os arquivos- multi thread (Jupyter)
#   (start = Sys.time())
#   future::plan(future::multicore)
#   invisible(future.apply::future_lapply(X   = hex_com_vizinhos$id,
#                                         FUN = gh_route_alt,
#                                         future.seed = TRUE))
#   Sys.time()
#   Sys.time() - start
  
#   # Remover arquivo inteiramente processado
#   file.remove(arq) 
  
#   # Limpar memória
#   rm(hex_com_vizinhos)  
#   gc(T)
  
# }

# Método 3

In [12]:
# # ------------------------------------------------------------------------------
# # Routing a partir de dois pontos com rotas alternativas (até 3 por par OD)
# # ------------------------------------------------------------------------------

# # Faz query de routing no GraphHopper e retorna resultados principais em dataframe,
# # com rotas até 3 alternativas por par OD - aceita um dataframe de uma linha como
# # entrada
# gh_route_alt <- function(hex_id, route_options) {
#   # url <- 'http://localhost:8989/route/?point=-23.5314933121698%2C-46.634354542765&point=-23.5390199310058%2C-46.6376369484305&profile=bike&instructions=false&calc_points=true&details=average_speed'
#   # url <- ods_vgs %>% slice(1) %>% select(url) %>% pull()
#   # Estação Vila Madalena: -23.546258,-46.690898
#   # IME: -23.559007,-46.73208
#   # CCSP: -23.571498,-46.639806
  
#   # df_line$url <- paste('http://localhost:8989/route/?point=', 
#   # '-23.541504', '%2C', '-46.685779', '&point=', 
#   # '-23.571498', '%2C', '-46.639806', route_options,
#   # sep = '')
  
#   # hex_id <- '89a81046b2fffff-89a81044d8fffff'
#   # hex_id <- hex_com_vizinhos %>% head(1) %>% select(id) %>% pull()
#   df_line <- hex_com_vizinhos %>% filter(id == hex_id)
  
#   # Fazer a GET de roteamento no Grahphopper
#   # print(df_line$url)
#   gh_response <- GET(df_line$url)
  
#   # Mensagem tem que ser "Success: (200) OK"
#   if (http_status(gh_response)$message == 'Success: (200) OK') {
    
#     # Resposta da query, já colapsada e transformada em dataframe
#     # Remover aviso de 'No encoding supplied: defaulting to UTF-8' na linha fromJSON()
#     suppressMessages(
#       response_text <- 
#         # Ignorar aviso 'argument is not an atomic vector; coercing'
#         suppressWarnings(str_c(content(gh_response, 'text'), collapse = ", ")) %>% 
#         # Concatenar toda a string de resultados
#         str_c("[", ., "]") %>% 
#         # Transformar em dataframe
#         fromJSON() %>% 
#         as.data.frame()
#     )
    
#     # Nos interessa a coluna de 'paths', como um novo dataframe
#     paths <- response_text$paths %>% as.data.frame()
    
#     # Puxar osm_way_ids dos resultados de cada alternativa e gravar em pasta separada
#     for (i in seq(1, length(paths$details$osm_way_id))) {
#       # i <- 1
#       osm_ways <- paths$details$osm_way_id[i] %>% as.data.frame()
      
#       if (nrow(osm_ways) > 0) {
#         # TODO: Inserir número da rota alternativa aqui
#         osm_ways <- osm_ways %>% mutate(index = row_number()) %>% select(index, osm_way_id = X3)
        
#         # Gravar resultados
#         osm_way_out <- sprintf('%s/%s_%i.csv', pasta_ids_aopt_19, df_line$id, i)
#         write_delim(osm_ways, osm_way_out, delim = ';')
        
#       } else {
#         # Se não há osm_way_ids, é porque os pontos estão muito próximos uns dos
#         # outros, mesmo que seja um osm_way_id diferente entre a origem e o 
#         # destino. A distância e a velocidade calculadas vão ser zero também - 
#         # pular este registro, que vai ser vazio
#         return(sprintf('Pulando: %s não tem osm_way_ids (provavelmente tem distância = 0)', hex_id))
#       }
      
#     }
    
    
#     # Isolar colunas de interesse
#     paths <- 
#       paths %>% 
#       # Calcular tempo em segundos e velocidade média
#       mutate(time = time / 1000,
#              speed = distance / time * 3.6) %>% 
#       # Descartar colunas extras - a coluna poly é o shape da rota traçada
#       select(distance, weight, time, speed, poly = points)
    
#     # Testar polyline:
#     # https://valhalla.github.io/demos/polyline/?unescape=true&polyline6=false#%0A
    
#     # Adicionar colunas de informação vindas do dataframe original
#     paths <- 
#       paths %>% 
#       mutate(hex_id    = df_line$id,
#              alt       = row_number(),
#              # id_hex.x  = df_line$id_hex_x,
#              # id_hex.y  = df_line$id_hex_y,
#              .before = 'distance') #%>% 
#       # mutate(lon.x     = df_line$lon.x,
#       #        lat.x     = df_line$lat.x,
#       #        lon.y     = df_line$lon.y,
#       #        lat.y     = df_line$lat.y,
#       #        .after = 'poly')
    
#   } else {
    
#     # Se a query no GraphHopper não deu resultados, guardar como dataframe vazio
#     paths <- data.frame(hex_id    = df_line$id,
#                         alt       = NA,
#                         # id_hex.x  = df_line$id_hex_x,
#                         # id_hex.y  = df_line$id_hex_y,
#                         distance  = NA,
#                         weight    = NA,
#                         time      = NA,
#                         speed     = NA,
#                         poly      = NA
#                         # lon.x     = df_line$lon.x,
#                         # lat.x     = df_line$lat.x,
#                         # lon.y     = df_line$lon.y,
#                         # lat.y     = df_line$lat.y
#     )
    
#   }
  
#   # Guardar resultados temporários
#   tmp_file <- sprintf('%s/%s_modalt.csv', pasta_rts_aopt_19, df_line$id)
#   write_delim(paths, tmp_file, delim = ';')

#   # Guardar ids já processados em arquivo próprio
#   df_line <- df_line %>% select(id)
#   ids_processados <- sprintf('%s/tmp_00_ids_processados.csv', pasta_aoprv_alter)
  
#   if (file.exists(ids_processados)) {
#     write_delim(df_line, ids_processados, delim = ';', append = TRUE)
#   } else if (!file.exists(ids_processados)) {
#     write_delim(df_line, ids_processados, delim = ';', append = FALSE)
#   }
  
# }

In [13]:
# detach("package:tidylog")

In [14]:
# # Arquivos a processar
# arqs <- list.files(pasta_aoprv_alter, pattern = '^tmp_base_para_routing_res09_26vizinhos_[0-9]{3}.csv', full.names = TRUE)
# # Arquivos já processados
# ids_processados <- sprintf('%s/tmp_00_ids_processados.csv', pasta_aoprv_alter)

# for (arq in arqs) {
#   # arq <- arqs[1]
#   print(arq)

#   # Abrir base para routing com 26 vizinhos, manter só id e url  
#   hex_com_vizinhos <- read_delim(arq, delim = ';', col_types = cols(.default = "c"))
  
#   # Checar quais resultados já foram rodados - abrir lista, puxar ids e remover
#   # do dataframe hex_com_vizinhos se houver
#   # library('tidylog')
#   if (file.exists(ids_processados)) {
#     arqs_resultados <- read_delim(ids_processados, delim = ';', col_types = "c")
#     hex_com_vizinhos <- hex_com_vizinhos %>% filter(!id %in% arqs_resultados$id)
#     rm(arqs_resultados)
#     nrow(hex_com_vizinhos)
#   }

#   # Se arquivo já foi completamente processado, removê-lo e passar para o seguinte
#   if (nrow(hex_com_vizinhos) == 0) { 
#       file.remove(arq) 
#       next
#   }

#   # Limpar memória
#   gc(T)
  
#   # Criar ttmatrix a partir do GrahHopper - melhor rodar no Jupyter se for AOP;
#   # se forem só as rotas originais, é ok rodar no RStudio
#   # for (id in hex_com_vizinhos$id) { gh_route_alt(id, route_options = route_options) }
#   # Rodar função para todos os arquivos- multi thread (Jupyter)
#   (start = Sys.time())
#   future::plan(future::multicore)
#   invisible(future.apply::future_lapply(X   = hex_com_vizinhos$id,
#                                         FUN = gh_route_alt,
#                                         future.seed = TRUE))
#   Sys.time()
#   Sys.time() - start
  
#   # Remover arquivo inteiramente processado
#   file.remove(arq) 

#   # Limpar memória
#   rm(hex_com_vizinhos)  
#   gc(T)
    
# }

# Método 1

In [15]:
# # Dividir dataframe em várias partes para rodar
# hex_com_vizinhos <- sprintf("%s/00_base_para_routing_res09_26vizinhos.csv", pasta_aoprv_alter)
# hex_com_vizinhos <- read_delim(hex_com_vizinhos, delim = ';', col_types = cols(.default = "c"))
# nrow(hex_com_vizinhos) # 10911566

# hex_com_vizinhos <- hex_com_vizinhos %>% slice(1:1000000)

In [16]:
# # Checar quais resultados já foram rodados - abrir lista, puxar ids e remover
# # do dataframe hex_com_vizinhos se houver
# # library('tidylog')
# arqs_resultados <- data.frame(arq = list.files(pasta_rts_aopt_19, recursive = FALSE, full.names = FALSE))
# arqs_resultados <- arqs_resultados %>% mutate(hex_id = str_replace(arq, '_modalt.csv', ''))
# hex_com_vizinhos <- hex_com_vizinhos %>% filter(!id %in% arqs_resultados$hex_id)
# rm(arqs_resultados)

# # Limpar memória
# gc(T)

# nrow(hex_com_vizinhos)

In [17]:
# detach("package:tidylog")

In [18]:
# # Criar ttmatrix a partir do GrahHopper - melhor rodar no Jupyter se for AOP;
# # se forem só as rotas originais, é ok rodar no RStudio
# # for (id in hex_com_vizinhos$id) { gh_route_alt(id, route_options = route_options) }
# # Rodar função para todos os arquivos- multi thread (Jupyter)
# (start = Sys.time())
# future::plan(future::multicore)
# invisible(future.apply::future_lapply(X   = hex_com_vizinhos$id,
#                                             FUN = gh_route_alt,
#                                             future.seed = TRUE))
# Sys.time()
# Sys.time() - start

# Método 2

In [19]:
# detach("package:tidylog")

In [20]:
# # Definir intervalo que custe pouca memória do future
# intervalo <- 50000
# # Puxar valor de nrow(hex_com_vizinhos)
# max_value <- 10911566

# # Abrir base hex_com_vizinhos e selecionar quantidade de linhas
# # de acordo com intervalo, checar ids que já rodaram, fazer 
# # routing com future, limpar memória e reiniciar até terminar
# for (i in seq(1, max_value, intervalo)) {
#   # i <- 1; 
  
#   # Se valor máximo pelo intervalo for maior que dataframe,
#   # considerar tamanho do dataframe como máximo
#   interval_max <- i + intervalo - 1
#   if (interval_max > max_value) { interval_max <- max_value}
#   print(sprintf('%i - %i', i, interval_max))
  
#   # Abrir base para routing com 26 vizinhos, manter só id e url
#   hex_com_vizinhos <- sprintf("%s/00_base_para_routing_res09_26vizinhos.csv", pasta_aoprv_alter)
#   hex_com_vizinhos <- read_delim(hex_com_vizinhos, delim = ';', col_types = cols(.default = "c"))
#   hex_com_vizinhos <- hex_com_vizinhos %>% select(id, url)
  
#   # Puxar só linhas correspondentes ao intervalo
#   hex_com_vizinhos <- hex_com_vizinhos %>% slice(i:interval_max)
  
#   # Checar quais resultados já foram rodados - abrir lista, puxar ids e remover
#   # do dataframe hex_com_vizinhos se houver
#   # library('tidylog')
#   arqs_resultados <- data.frame(arq = list.files(pasta_rts_aopt_19, recursive = FALSE, full.names = FALSE))
#   arqs_resultados <- arqs_resultados %>% mutate(hex_id = str_replace(arq, '_modalt.csv', ''))
#   hex_com_vizinhos <- hex_com_vizinhos %>% filter(!id %in% arqs_resultados$hex_id)
#   rm(arqs_resultados)
#   # nrow(hex_com_vizinhos)

#   # Limpar memória
#   gc(T)
  
#   # Criar ttmatrix a partir do GrahHopper - melhor rodar no Jupyter se for AOP;
#   # se forem só as rotas originais, é ok rodar no RStudio
#   # for (id in hex_com_vizinhos$id) { gh_route_alt(id, route_options = route_options) }
#   # Rodar função para todos os arquivos- multi thread (Jupyter)
#   (start = Sys.time())
#   future::plan(future::multicore)
#   invisible(future.apply::future_lapply(X   = hex_com_vizinhos$id,
#                                         FUN = gh_route_alt,
#                                         future.seed = TRUE))
#   Sys.time()
#   Sys.time() - start
  
#   # Limpar memória
#   rm(hex_com_vizinhos)
#   gc(T)
    
# }